In [18]:
# import matplotlib.pyplot as plt
# import numpy as np
# import tensorflow as tf

# from tqdm import tqdm

In [4]:
# from tensorflow.keras.utils import image_dataset_from_directory

# img_height = 224
# img_width = 224
# batch_size = 32

# train_ds = image_dataset_from_directory(
#     "C:/Users/vinee/Downloads/Monkeypox Skin Image Dataset",
#     label_mode='categorical',
#     validation_split=0.2,
#     subset="training",
#     seed=123,
#     image_size=(img_height, img_width),
#     batch_size=batch_size)

# val_ds = image_dataset_from_directory(
#     "C:/Users/vinee/Downloads/Monkeypox Skin Image Dataset",
#     label_mode='categorical',
#     validation_split=0.2,
#     subset="validation",
#     seed=123,
#     image_size=(img_height, img_width),
#     batch_size=batch_size)

Found 770 files belonging to 1 classes.
Using 616 files for training.
Found 770 files belonging to 1 classes.
Using 154 files for validation.


In [14]:
# class_names = train_ds.class_names
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[np.argmax(labels[i])])
#     plt.axis("off")

In [20]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation
from keras.layers.core import Flatten, Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import optimizers
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
%matplotlib inline

In [23]:
datagen = ImageDataGenerator(rescale=1./255)
train_path = 'C:/Users/vinee/Downloads/Monkeypox Skin Image Dataset/Monkeypox Skin Image Dataset'

train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(115,115), 
                                                         classes=['Monkeypox','Non Monkeypox'], class_mode='categorical', 
                                                         batch_size=27560)

imgs, label = next(train_batches)
print(label.shape)
print(imgs.shape)

Found 279 images belonging to 2 classes.
(279, 2)
(279, 115, 115, 3)


In [24]:
X_train, X_test, y_train, y_test = train_test_split(imgs,label,test_size=0.3,random_state=42)

In [25]:
datagen = ImageDataGenerator(
        rotation_range=10,  # Rotate images randomly 
        zoom_range = 0.1, # Zoom images randomly
        width_shift_range=0.1,  # Horizontally shift images in random order
        height_shift_range=0.1  # Vertically shift images in random order
)

datagen.fit(X_train)

In [27]:
# for i in range (0,5):
#     image = imgs[i]
#     plt.imshow((image * 255).astype(np.uint8))
#     if (label[i][0]==1):
#       print("Monkeypox")
#     if (label[i][1]==1):
#         print("Non Monkeypox")
#     plt.show()

In [28]:
def show_confusion_matrix(history,model, x_test, y_test):
      
  # summarize history for accuracy
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'],loc='upper left')
  plt.show()

  pre_cls=model.predict_classes(x_test)   

  #Decode from one hot encoded to original for confusion matrix
  decoded_label = np.zeros(x_test.shape[0],)
  for i in range(y_test.shape[0]):
    decoded_label[i] = np.argmax(y_test[i])
  #print(decoded_label.shape)

  cm1 = confusion_matrix(decoded_label,pre_cls)
  print('Confusion Matrix : \n')
  print(cm1)

In [29]:
image_w, image_h = 115, 115
model = keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(image_w, image_h, 3))

model.summary()    
    # Adding custom layers to create a new model 
new_model2 = Sequential([
    model,
    Flatten(name='flatten'),
    Dense(512, activation='relu', name='new_fc1'),
    Dropout(0.5),
    Dense(2, activation='softmax', name='new_predictions')
])

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 115, 115, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 121, 121, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 58, 58, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 58, 58, 64)   256         ['conv1_conv[0][0]']      

In [30]:
new_model2.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.00001), metrics=["accuracy"])
es = keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1, mode='max', patience=2)

c:\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [31]:
batch_size = 100
history = new_model2.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=10, validation_data = (X_test, y_test), verbose = 1, steps_per_epoch=X_train.shape[0] // batch_size, callbacks=[es])

print("Batch Size: 20, Epoch: 5, Optimizer: Adam")

show_confusion_matrix(history,new_model2, X_test, y_test)

C:\Users\vinee\AppData\Local\Temp\ipykernel_7740\4189082616.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = new_model2.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=10, validation_data = (X_test, y_test), verbose = 1, steps_per_epoch=X_train.shape[0] // batch_size, callbacks=[es])


Epoch 1/10
1/1 [==============================] - 54s 54s/step - loss: 0.3047 - accuracy: 0.8737 - val_loss: 0.0192 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 24s 24s/step - loss: 0.1089 - accuracy: 0.9579 - val_loss: 0.0034 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 24s 24s/step - loss: 0.0309 - accuracy: 0.9900 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 23s 23s/step - loss: 0.0439 - accuracy: 0.9800 - val_loss: 6.2670e-04 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 24s 24s/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 3.5041e-04 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 22s 22s/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 2.1421e-04 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 22s 22s/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 1.3843e-04 - val_accuracy: 1.0000
Epoch 8/10


KeyError: 'acc'